# Bix Tecnologia


Fonte: https://docs.google.com/spreadsheets/d/1Sc6hN48b-lWbDdYNd_t9j8Bt0EZ8BUNG/edit?pli=1#gid=693918541

# Initial imports

In [1]:
%pip install ipython-autotime  --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive, files
import pandas as pd
drive.mount('/content/drive', force_remount=True)
%load_ext autotime

Mounted at /content/drive
time: 230 µs (started: 2023-07-07 23:42:36 +00:00)


In [20]:
clients = pd.read_csv("/content/drive/MyDrive/datasets/bix-tecnologia/clients.csv", encoding='utf-8')
clients['DateOfBirth'] = pd.to_datetime(clients['DateOfBirth'], format="%m/%d/%Y")

products  = pd.read_csv("/content/drive/MyDrive/datasets/bix-tecnologia/products.csv", encoding='utf-8')

sales = pd.read_csv("/content/drive/MyDrive/datasets/bix-tecnologia/sales.csv", encoding='utf-8', skiprows=4)
sales = sales[['ID', 'StoreID', 'ProductID', 'ClientID','Discount', 'UnitPrice', 'Quantity', 'Date']]
sales['Date'] = pd.to_datetime(sales['Date'], format="%m/%d/%Y")

stores    = pd.read_csv("/content/drive/MyDrive/datasets/bix-tecnologia/stores.csv", encoding='utf-8')

time: 99.9 ms (started: 2023-07-07 23:59:04 +00:00)


Vou renomear as colunas para ficar mais legível e poder unir os dataframes com a função merge() formando apenas um dataset.

In [21]:
clients = clients.rename(columns={'ID':'client_id', 'City':'client_city', 'State':'client_state', 'DateOfBirth':'client_birth', 'Sex':'client_sex'})
products = products.rename(columns={'ID':'product_id', 'Name':'product_name', 'Size':'product_size'})
sales = sales.rename(columns={'ID':'id', 'ProductID':'product_id', 'ClientID':'client_id', 'Discount':'discount', 'UnitPrice':'unit_price', 'Quantity':'quantity', 'StoreID':'store_id', 'Date':'date'})
stores = stores.rename(columns={'ID':'store_id', 'Name':'store_name', 'State':'store_state'})

time: 3.14 ms (started: 2023-07-07 23:59:07 +00:00)


Agora, vou criar um único dataframe com todos os arquivos. Além disso, vou reordenar as colunas para legibilidade.

In [22]:
dataset = sales.merge(stores, on='store_id')
dataset = dataset.merge(products, on='product_id')
dataset = dataset.merge(clients, on='client_id')

#dataset = dataset.drop(['id', 'store_id', 'product_id', 'client_id'], axis=1)
dataset = dataset[['date', 'store_name', 'store_state', 'product_name', 'product_size', 'client_city', 'client_state', 'client_birth', 'client_sex', 'discount', 'unit_price', 'quantity', 'id', 'store_id', 'product_id', 'client_id']]

time: 36.1 ms (started: 2023-07-07 23:59:09 +00:00)


# EDA

In [23]:
import numpy as np

time: 300 µs (started: 2023-07-07 23:59:14 +00:00)


In [24]:
dataset.head()

,date,store_name,store_state,product_name,product_size,client_city,client_state,client_birth,client_sex,discount,unit_price,quantity,id,store_id,product_id,client_id
0,2018-12-10,Curitiba,PR,Tempestade,G,Curitiba,PR,1985-06-28,Homem,"0,08","249,2",1,80260d682079b6090c8285b398c50d97,4,002552c0663708129c0019cc97552d7d3,14001
1,2019-04-25,Curitiba,PR,Thanos,G,Curitiba,PR,1985-06-28,Homem,"0,1","162,4",1,3275736da1234f55d52bdf09d86b93bc,4,001b237c0e9bb435f2e54071129237e93,14001
2,2019-07-17,Curitiba,PR,Capitão América,P,Curitiba,PR,1985-06-28,Homem,"0,1","194,6",1,f3513eec8f1434b5b7e04ddf3598df40,4,00066f42aeeb9f3007548bb9d3f33c381,14001
3,2018-12-03,Curitiba,PR,Tempestade,G,Curitiba,PR,1987-10-21,Homem,"0,08","249,2",1,c330b70c5e60bf56e90ebe09e045f79e,4,002552c0663708129c0019cc97552d7d3,14116
4,2018-04-29,Curitiba,PR,Bazinga,G,Curitiba,PR,1987-10-21,Homem,"0,1","218,4",2,eced501ea6ea98469acff0ccbf3ca09b,4,002ec297b1b00fb9dde7ee6ac24b67713,14116


time: 11.1 ms (started: 2023-07-07 23:59:16 +00:00)


In [25]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32753 entries, 0 to 32752
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          32753 non-null  datetime64[ns]
 1   store_name    32753 non-null  object        
 2   store_state   32753 non-null  object        
 3   product_name  32753 non-null  object        
 4   product_size  32753 non-null  object        
 5   client_city   32753 non-null  object        
 6   client_state  32753 non-null  object        
 7   client_birth  32753 non-null  datetime64[ns]
 8   client_sex    32753 non-null  object        
 9   discount      32753 non-null  object        
 10  unit_price    32753 non-null  object        
 11  quantity      32753 non-null  int64         
 12  id            32753 non-null  object        
 13  store_id      32753 non-null  int64         
 14  product_id    32753 non-null  object        
 15  client_id     32753 non-null  int64 

Os atributos 'discount' e 'unit_price' são números, mas estão definidos como 'object'. Isso acontece porque os números contém vírgula e não ponto. Vou mudar seus tipos para que eu possa criar os gráficos de maneira adequada.

In [26]:
dataset['discount']   = np.vectorize(lambda x: x.replace(',', '.'))(dataset['discount'])
dataset['unit_price'] = np.vectorize(lambda x: x.replace(',', '.'))(dataset['unit_price'])
dataset = dataset.astype({'discount': 'float64'})
dataset = dataset.astype({'unit_price': 'float64'})

time: 39.8 ms (started: 2023-07-07 23:59:22 +00:00)


## Data visualization

# Train/Test split

# Feature engineering

# Model training